# OpenAPI.NET Notebook Sample

In this sample we will use OpenAPI.NET package to access cTrader Open API, and show a trading account real time stats.

Let's start by installing the Nuget package:

In [ ]:
#r "nuget: Spotware.OpenAPI.Net"

Then we will add the OpenAPI.NET usings:

In [ ]:
using OpenAPI.Net;
using OpenAPI.Net.Auth;
using OpenAPI.Net.Helpers;

Please create a credentials.json file in current directory of notebook file and fill it with your Open API credentials:

```json
{
  "ClientId": "your app client id",
  "Secret": "your app secret",
  "HostType": "your account type (demo or live)",
  "AccessToken": "your access token, create one by using your app playground on open API site",
  "AccountId": "Your account ID as a number without quotes"
}
```

We will load your created credentials file:

In [ ]:
using System.Text.Json;
using System.Text.Json.Serialization;
using System.IO;

public record Credentials(string ClientId, string Secret, string HostType, string AccessToken, long AccountId);

private async Task<Credentials> GetCredentials()
{
    using var stream = File.OpenRead("credentials.json");
    return  await JsonSerializer.DeserializeAsync<Credentials>(stream);
}

var credentials = await GetCredentials();

Console.WriteLine(credentials);

We got the API credentials, now lets create an OpenAPI.NET client, and authorize our app and account:

In [ ]:
using System.Reactive.Linq;
using Google.Protobuf;

var host = credentials.HostType.Equals("demo", StringComparison.OrdinalIgnoreCase) ? ApiInfo.DemoHost : ApiInfo.LiveHost;

var client = new OpenClient(host, ApiInfo.Port, TimeSpan.FromSeconds(15));

client.Where(iMessage => iMessage is not ProtoHeartbeatEvent).Subscribe(message => Console.WriteLine($"Message Received: {message}"), exception => Console.WriteLine($"Exception: {exception}"));

private async void OnAppAuthorized(ProtoOAApplicationAuthRes response)
{
    var accountAuthReq = new ProtoOAAccountAuthReq
    {
        CtidTraderAccountId = credentials.AccountId,
        AccessToken = credentials.AccessToken
    };

    client.OfType<ProtoOAAccountAuthRes>().Subscribe(OnAccountAuthRes);

    await client.SendMessage(accountAuthReq, accountAuthReq.PayloadType);
}


private void OnAccountAuthRes(ProtoOAAccountAuthRes response)
{
    Console.WriteLine("OnAccountAuthRes");
}

client.OfType<ProtoOAApplicationAuthRes>().Subscribe(OnAppAuthorized);

await client.Connect();

var applicationAuthReq = new ProtoOAApplicationAuthReq
{
    ClientId = credentials.ClientId,
    ClientSecret = credentials.Secret,
};

await client.SendMessage(applicationAuthReq, applicationAuthReq.PayloadType);

// We will wait 10 seconds to receive the responses
await Task.Delay(10000);